### Imports

In [28]:
import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

### Load data

In [2]:
train = pd.read_csv("../../data/home_default/train.csv")
test  = pd.read_csv("../../data/home_default/test.csv")

print("Shape of train:", train.shape)
print("Shape of test:",  test.shape)

Shape of train: (307511, 122)
Shape of test: (48744, 121)


### Merge train and test data

In [3]:
full = pd.concat([train, test])
train_N = len(train)

### Fill categorical NaN

In [4]:
for col in ('NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'):
    full[col] = full[col].fillna(full[col].mode()[0])

### Fill numeric NaN

In [5]:
missing_cols = [col for col in full.columns if any(full[col].isnull())]
for col in missing_cols:
    full[col] = full[col].fillna(full[col].median())

### Confirm missing values are gone

In [6]:
full.isnull().sum().sum()

0

### Get dummies

In [7]:
full = pd.get_dummies(full)

### Separate data back into train and test

In [8]:
train = full[:train_N]
test = full[train_N:]
del full, train_N

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,1,0,0,0,0,0,0,0,1
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1,0,0,0,0,0
3,29686.5,312682.5,297000.0,135000.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1,0,0,0,0,0,0,0,0,1
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,1,0,0


### Split again into predictors, target, and id

In [9]:
train_y = train.TARGET
train_x = train.drop(["TARGET", "SK_ID_CURR"], axis=1)

test_id = test.SK_ID_CURR
test_x  = test.drop(["TARGET", "SK_ID_CURR"], axis=1)

### Imports and scoring helper function

In [33]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold

kfold = KFold(n_splits=2)
    
def rmsle_cv(model):
    rmse= np.sqrt(-cross_val_score(model, train_x, train_y, cv=kfold, scoring="neg_mean_squared_error"))
    print(rmse)
    print()
    print(sum(rmse) / len(rmse))

## LGBM Regressor

In [34]:
lgbm_model = LGBMRegressor()

start = time.time()
lgbm_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 14 seconds


### Scoring

In [35]:
start = time.time()
rmsle_cv(lgbm_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.26223286  0.26024019]

0.261236525624

Scoring took 21 seconds


## Linear Regressor

In [23]:
lin_model = LinearRegression()

start = time.time()
lin_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

[ 0.26395911  0.26393661  0.26354644]

0.263814054337


### Scoring

In [ ]:
start = time.time()
rmsle_cv(lin_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

## Random Forest Regressor

WARNING: Takes a lot of time

In [24]:
rf_model = RandomForestRegressor()

start = time.time()
rf_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

[ 0.27978732  0.28013387  0.28058117]

0.280167454734


### Scoring

In [ ]:
start = time.time()
rmsle_cv(rf_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

## Logistic Regression

In [36]:
log_model = LogisticRegression()

start = time.time()
log_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 15 seconds


### Scoring

In [ ]:
start = time.time()
rmsle_cv(log_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-37-4cbf33d32a3f>", line 2, in <module>
    rmsle_cv(rf_model)
  File "<ipython-input-33-e0ccba46f9ba>", line 10, in rmsle_cv
    rmse= np.sqrt(-cross_val_score(model, train_x, train_y, cv=kfold, scoring="neg_mean_squared_error"))
  File "/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 342, in cross_val_score
    pre_dispatch=pre_dispatch)
  File "/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 206, in cross_validate
    for train, test in cv.split(X, y, groups))
  File "/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 779, in __call__
    while self.dispatch_one_batch(iterator):
  File "/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", lin

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 637, in _abort_queues
    self._abort_queue(stream)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 662, in _abort_queue
    poll

## Lasso Regression

In [ ]:
las_model = Lasso()

start = time.time()
las_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

### Scoring

In [ ]:
start = time.time()
rmsle_cv(las_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

## Ridge Regression

In [ ]:
rid_model = Ridge()

start = time.time()
rid_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

rmsle_cv(rid_model)

### Scoring

In [ ]:
start = time.time()
rmsle_cv(rid_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

## XGB Regressor

In [1]:
xgb_model = XGBRegressor()
xgb_model.fit(train_x, train_y)
score_model(xgb_model)

### Scoring

In [ ]:
start = time.time()
rmsle_cv(xgb_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

### Get predictions

In [14]:
predictions = lgbm_model.predict(test_x)

### Restrict predictions to appropriate range

In [21]:
predictions = np.clip(predictions, 0, 1)

# Sanity check
any(predictions < 0) or any(predictions > 1)

### Save file to CSV

In [25]:
pd.DataFrame({
    "SK_ID_CURR": test_id,
    "TARGET": predictions
}).to_csv("../../submissions/lgbm_regressor.csv", index=False)